In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D    
import random
import pickle
import cv2

In [0]:
random.seed(0)

our data in /kaggle/input are pickle files, so we should unpickle them

In [0]:
!ls /kaggle/input/traffic-signs-preprocessed

In [0]:
with open('/kaggle/input/traffic-signs-preprocessed/train.pickle','rb') as f:
    train_data = pickle.load(f)

with open('/kaggle/input/traffic-signs-preprocessed/valid.pickle','rb') as f:
    valid_data = pickle.load(f)

with open('/kaggle/input/traffic-signs-preprocessed/test.pickle','rb') as f:
    test_data = pickle.load(f)

type(train_data)

In [0]:
X_train, y_train = train_data['features'], train_data['labels']
X_valid, y_valid = valid_data['features'], valid_data['labels']
X_test, y_test = test_data['features'], test_data['labels']
print("X train shape:",X_train.shape)
print("X train shape:",X_valid.shape)
print("X train shape:",X_test.shape)

In [0]:
print(X_train[0].shape == X_train[0].shape)
print(X_valid[0].shape == X_valid[0].shape)
print(y_test[0].shape == y_test[0].shape)


In [0]:
data = pd.read_csv("/kaggle/input/traffic-signs-preprocessed/label_names.csv")



In [0]:
num_of_samples = []
cols = 5
num_classes = 43
fig, axs = plt.subplots(nrows=num_classes,ncols=cols,figsize=(5,50))
fig.tight_layout()

for i in range(cols):
    for j,row in data.iterrows():
        x_selected = X_train[y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0,(len(x_selected)-1)),: ,:],cmap=plt.get_cmap('gray'))
        axs[j][i].axis("off")
        if i ==2:
            axs[j][i].set_title(str(j)+'-'+ row["SignName"])
            num_of_samples.append(len(x_selected))

In [0]:
print(num_of_samples)
plt.figure(figsize=(12,4))
plt.bar(range(0,num_classes),num_of_samples)
plt.title("Distribution of the train dataset")
plt.xlabel("Class of number")
plt.ylabel("Number of images")

In [0]:
plt.imshow(X_train[123])
plt.axis("off")
print(X_train[123].shape)

In [0]:
def grayscale(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img
def equalize(img):
    img = cv2.equalizeHist(img)
    return img
def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img
    return img

#list(map(preprocessing,X_train))

In [0]:
X_train = np.array(list(map(preprocessing,X_train)),dtype=np.uint8)
X_valid = np.array(list(map(preprocessing,X_valid)),dtype=np.uint8)
X_test = np.array(list(map(preprocessing,X_test)),dtype=np.uint8)

In [0]:
plt.imshow(X_train[random.randint(0,len(X_train)-1)], cmap=plt.get_cmap('gray'))
plt.axis("off")


In [0]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)


In [0]:

X_train = X_train.reshape(34799, 32, 32,1)
X_valid = X_valid.reshape(4410, 32, 32, 1)
X_test = X_test.reshape(12630, 32, 32, 1)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

In [0]:
def model():
    model = Sequential()
    model.add(Conv2D(26,(5,5),input_shape=(32,32,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(15,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(600,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(Adam(lr=0.01),loss='categorical_crossentropy',metrics=[])
    return model

In [0]:
model = model()
model.summary()

In [0]:
model.fit(X_train,y_train,batch_size=400, epochs=10, validation_data=(X_valid,y_valid),verbose=1,shuffle=1)